In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [2]:
data = pd.read_csv('WMT1.csv')
data_17 = data[data.Year == 2017]

# calculate the mean std
mean_17 = data_17.groupby('Week_Number')['Return'].mean().tolist()
std_17 = data_17.groupby('Week_Number')['Return'].std().tolist()
week_id_17 = list(data_17.groupby('Week_Number').indices.keys())
color_17 = data_17.groupby('Week_Number')['label'].agg(lambda x: x.mode()).tolist()

data_18 = data[data.Year == 2018]
data_18.drop(data_18.index[-1], inplace=True)

# calculate the mean std
mean_18 = data_18.groupby('Week_Number')['Return'].mean().tolist()
std_18 = data_18.groupby('Week_Number')['Return'].std().tolist()
weekly_18 = data[(data.Year == 2018) & (data.Weekday == 'Friday')]['Adj Close'].values  # select Friday's data

week_id_18 = list(data_18.groupby('Week_Number').indices.keys())
color_18 = data_18.groupby('Week_Number')['label'].agg(lambda x: x.mode()).tolist()


/Users/wangxiaoyang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [3]:
df_17 = pd.DataFrame(
    {
        'id':week_id_17,
        'mean':mean_17,
        'std':std_17,
        'label':color_17
    },
    columns = ['id','mean','std','label']
)

df_18 = pd.DataFrame(
    {
        'id':week_id_18,
        'mean':mean_18,
        'std':std_18,
        'label':color_18
    },
    columns = ['id','mean','std','label']
)

In [4]:
x_train = df_17[['mean','std']].values
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
y_train= df_17['label'].values

x_test = df_18[['mean','std']].values
scaler = StandardScaler()
scaler.fit(x_test)
x_test = scaler.transform(x_test)
y_test = df_18['label'].values

In [11]:
svm_classifier = svm.SVC(kernel ='linear')
svm_classifier.fit(x_train,y_train)
predicted = svm_classifier.predict(x_test)
accuracy = svm_classifier.score(x_test,y_test)
accuracy # question1

0.9038461538461539

In [12]:
#question2
a = confusion_matrix(y_test,predicted)
a

array([[31,  0],
       [ 5, 16]])

In [13]:
#question3
TPR = a[0][0]/(a[0][0]+a[0][1])
TNR = a[1][1]/(a[1][0]+a[1][1])
print('true positive rate is :',TPR)
print('true negative rate is :',TNR)

true positive rate is : 1.0
true negative rate is : 0.7619047619047619


In [14]:
svm_classifier = svm.SVC(kernel ='rbf')
svm_classifier.fit(x_train,y_train)
prediction = svm_classifier.predict(x_test)
accuracy = svm_classifier.score(x_test,y_test)
accuracy # question4

0.9038461538461539

In [16]:
print('its accuracy is same as linear SVM')

its accuracy is same as linear SVM


In [17]:
svm_classifier = svm.SVC(kernel ='poly', degree = 2)
svm_classifier.fit(x_train,y_train)
prediction = svm_classifier.predict(x_test)
accuracy = svm_classifier.score(x_test,y_test)
accuracy # question5

0.6153846153846154

In [18]:
print('its accuracy worse than linear SVM')

its accuracy worse than linear SVM


In [19]:
#question4
previous_shares = 0
previous_balance = 100
weekly_balance = []
for i in range(len(weekly_18)-1):
    if predicted[i+1] == 'green':
        if previous_shares ==0 :
            previous_shares = previous_balance/weekly_18[i]
    elif predicted[i+1] == 'red':
        if previous_shares != 0:
            previous_balance = previous_shares*weekly_18[i]
            previous_shares = 0

    weekly_balance.append(previous_balance)
result1 = weekly_balance[-1]

In [20]:
result2 = 100/weekly_18[0]*weekly_18[-1]

print('result based on my labels: %f $'%result1)
print('result based on buy and hold: %f $'%result2)
print('my labels results in a large amount at the end of the year')

result based on my labels: 122.844823 $
result based on buy and hold: 94.188504 $
my labels results in a large amount at the end of the year
